In [ ]:
import os
import sys

from dotenv import find_dotenv, load_dotenv

if os.path.dirname(find_dotenv()) not in sys.path:
    sys.path.append(os.path.dirname(find_dotenv()))

from src.utils import debug as debug_util
from src.utils import llm as llm_util

load_dotenv(override=True)
VERBOSE = debug_util.string_to_bool(os.environ.get("VERBOSE"))
model = "anthropic.claude-3-sonnet-20240229-v1:0"

In [ ]:
llm = llm_util.create_llm("aws-redrock", model, verbose=VERBOSE, temperature=0)
# llm = llm_util.create_llm("anthropic", "claude-3-haiku-20240307", verbose=VERBOSE, temperature=0)

In [ ]:
question = "Give me some information of AWS Community Day New Zealand 2024?"
response = llm.invoke(question)

In [ ]:
print(response.content)

## Agent: Use Tools

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=1)

In [ ]:
search.invoke(question)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

tools = [search]
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=VERBOSE)

response = agent_executor.invoke({"input": question})

In [ ]:
response

In [ ]:
print(response["output"][0]["text"])